In [1]:
import os
import zipfile
from pathlib import Path
import warnings
from shutil import rmtree
import time

import pandas as pd
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm

import numpy as np
import SimpleITK as sitk

from surface_distance import compute_surface_distances, compute_robust_hausdorff


def compute_segmentation_scores(y_true, y_pred, spacing):
    if np.sum(y_pred) == 0:
        return {
            "dice_score": 0,
            "hausdorff_distance_95": 0,
            "recall": 0,
            "precision": 0,
            "contain": True
        }
    else:
        fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))
        tp = np.sum(np.logical_and(y_true, y_pred))
        fn = np.sum(np.logical_and(np.logical_not(y_pred), y_true))
        # tn = np.sum(
        #     np.logical_and(np.logical_not(y_true), np.logical_not(y_pred)))
        recall = tp / (tp + fn)
        precision = tp / (tp + fp)
        return {
            "dice_score":
            dice(y_true, y_pred),
            "hausdorff_distance_95":
            robust_hausdorff(y_true != 0, y_pred != 0, spacing, percent=95),
            "recall":
            recall,
            "precision":
            precision,
            "contain": True
        }


def robust_hausdorff(image0, image1, spacing, percent=95.0):
    surface_distances = compute_surface_distances(image0 != 0, image1 != 0,
                                                  spacing)
    return compute_robust_hausdorff(surface_distances, percent)


def dice(y_true, y_pred):
    return 2 * np.sum(np.logical_and(
        y_true, y_pred)) / (np.sum(y_true) + np.sum(y_pred))


def get_np_volume_from_sitk(sitk_image):
    trans = (2, 1, 0)
    pixel_spacing = sitk_image.GetSpacing()
    image_position_patient = sitk_image.GetOrigin()
    np_image = sitk.GetArrayFromImage(sitk_image)
    np_image = np.transpose(np_image, trans)
    return np_image, pixel_spacing, image_position_patient


class AIcrowdEvaluator:
    def __init__(
        self,
        ground_truth_segmentation_folder="/mnt/faststorage/jintao/HNSCC/hecktor2021_train/hecktor_nii/",
        bounding_boxes_file="/mnt/faststorage/jintao/HNSCC/hecktor2021_train/hecktor2021_bbox_training.csv",
        extraction_folder="C:/Users/ngochuyn/Documents/mnt/SCRATCH/ngochuyn/nnUNet_trained_models/nnUNet/3d_fullres/Task602_Hecktor_PET5/nnUNetTrainerV2__nnUNetPlansv2.1/fold_4/resampled/",
        round_number=1,
    ):
        """Evaluator for the Hecktor Challenge
        Args:
            ground_truth_folder (str): the path to the folder
                                       containing the ground truth segmentation.
            ground_truth_survival_file (str): the path to the file
                                       containing the ground truth survival time.
            bounding_boxes_file (str): the path to the csv file which defines
                                           the bounding boxes for each patient.
            extraction_folder (str, optional): the path to the folder where the
                                               extraction of the .zip submission
                                               will take place. Defaults to "data/tmp/".
                                               This folder has to be created beforehand.
            round_number (int, optional): the round number. Defaults to 1.
        """
        self.groud_truth_folder = Path(ground_truth_segmentation_folder)
        self.round = round_number
        self.extraction_folder = Path(extraction_folder)
        self.bounding_boxes_file = Path(bounding_boxes_file)

    def _evaluate_segmentation(self):

        groundtruth_paths = [
            f for f in self.groud_truth_folder.rglob("*_gtvt.nii.gz")
        ]
        bb_df = pd.read_csv(str(
            self.bounding_boxes_file.resolve())).set_index("PatientID")

        results_df = pd.DataFrame()

        missing_patients = list()
        unresampled_patients = list()
        resampler = sitk.ResampleImageFilter()
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        for path in tqdm(groundtruth_paths):
            patient_id = path.name[:7]
            prediction_files = [
                f
                for f in self.extraction_folder.rglob(patient_id + "*.nii.gz")
            ]
            if len(prediction_files) > 1:
                raise Exception(
                    "There is too many prediction files for patient {}".format(
                        patient_id))
            elif len(prediction_files) == 0:
                results_df = results_df.append(
                    {
                        "dice_score": 0,
                        "hausdorff_distance_95": np.inf,
                        "recall": 0,
                        "precision": 0,
                        'contain': False
                    },
                    ignore_index=True)

                missing_patients.append(patient_id)
                continue

            bb = np.array([
                bb_df.loc[patient_id, "x1"], bb_df.loc[patient_id, "y1"],
                bb_df.loc[patient_id, "z1"], bb_df.loc[patient_id, "x2"],
                bb_df.loc[patient_id, "y2"], bb_df.loc[patient_id, "z2"]
            ])

            image_gt = sitk.ReadImage(str(path.resolve()))
            image_pred = sitk.ReadImage(str(prediction_files[0].resolve()))

            resampler.SetReferenceImage(image_gt)
            resampler.SetOutputOrigin(bb[:3])

            voxel_spacing = np.array(image_gt.GetSpacing())
            output_size = np.round(
                (bb[3:] - bb[:3]) / voxel_spacing).astype(int)
            resampler.SetSize([int(k) for k in output_size])

            # Crop to the bonding box and/or resample to the original spacing
            spacing = image_gt.GetSpacing()
            if spacing != image_pred.GetSpacing():
                unresampled_patients.append(patient_id)

            image_gt = resampler.Execute(image_gt)
            image_pred = resampler.Execute(image_pred)

            results_df = results_df.append(
                compute_segmentation_scores(
                    sitk.GetArrayFromImage(image_gt),
                    sitk.GetArrayFromImage(image_pred),
                    spacing,
                ),
                ignore_index=True,
            )

        results_df = results_df[results_df['contain'] == True]

        _result_object = {
            "dice_score": results_df["dice_score"].mean(),
            "hausdorff_distance_95":
            results_df["hausdorff_distance_95"].median(),
            "recall": results_df["recall"].mean(),
            "precision": results_df["precision"].mean(),
        }

        print(_result_object)
        messages = list()
        if len(unresampled_patients) > 0:
            messages.append(
                f"The following patient(s) was/were not resampled back"
                f" to the original resolution: {unresampled_patients}."
                f"\nWe applied a nearest neighbor resampling.\n")

        if len(missing_patients) > 0:
            messages.append(
                f"The following patient(s) was/were missing: {missing_patients}."
                f"\nA score of 0 and infinity were attributed to them "
                f"for the dice score and Hausdorff distance respectively.")
        _result_object["message"] = "".join(messages)
        print(_result_object)
        return _result_object


for extraction_folder in ['/mnt/faststorage/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task226_hecktor_sine/nnUNetTrainerV2__nnUNetPlansv2.1/cv_niftis_postprocessed/', '/mnt/faststorage/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task229_hecktor_base_focal/nnUNetTrainerV2__nnUNetPlansv2.1/cv_niftis_postprocessed/', '/mnt/faststorage/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task222_hecktor/nnUNetTrainerV2__nnUNetPlansv2.1/cv_niftis_postprocessed/']:
    print(extraction_folder)
    AIcrowdEvaluator(
        extraction_folder=extraction_folder)._evaluate_segmentation()


  0%|          | 0/224 [00:00<?, ?it/s]

/mnt/faststorage/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task226_hecktor_sine/nnUNetTrainerV2__nnUNetPlansv2.1/cv_niftis_postprocessed/


  0%|          | 0/224 [00:00<?, ?it/s]

{'dice_score': 0.7595757072250091, 'hausdorff_distance_95': 2.9296860694885254, 'recall': 0.7886534292472428, 'precision': 0.7895430298585824}
{'dice_score': 0.7595757072250091, 'hausdorff_distance_95': 2.9296860694885254, 'recall': 0.7886534292472428, 'precision': 0.7895430298585824, 'message': "The following patient(s) was/were not resampled back to the original resolution: ['CHUS085', 'CHUS030', 'CHGJ087', 'CHUP012', 'CHGJ074', 'CHUS076', 'CHGJ081', 'CHUM019', 'CHUS045', 'CHUS048', 'CHUS028', 'CHGJ018', 'CHUP009', 'CHUM001', 'CHMR020', 'CHMR014', 'CHUM022', 'CHUS064', 'CHGJ034', 'CHUP008', 'CHUM030', 'CHGJ090', 'CHUS081', 'CHUS087', 'CHUS056', 'CHGJ058', 'CHUS027', 'CHUP010', 'CHGJ028', 'CHUP007', 'CHUM036', 'CHGJ035', 'CHUM008', 'CHMR013', 'CHMR016', 'CHGJ008', 'CHMR005', 'CHUP003', 'CHUS049', 'CHUS077', 'CHUS007', 'CHMR024', 'CHUM060', 'CHUP021', 'CHGJ017', 'CHUM024', 'CHUM065', 'CHGJ048', 'CHGJ015', 'CHUM047', 'CHUM017', 'CHUS039', 'CHUM064', 'CHUM058', 'CHUS041', 'CHGJ073', 'CHU

  0%|          | 0/224 [00:00<?, ?it/s]

{'dice_score': 0.7616391310550885, 'hausdorff_distance_95': 2.9296860694885254, 'recall': 0.7929149617191463, 'precision': 0.7870110592469598}
{'dice_score': 0.7616391310550885, 'hausdorff_distance_95': 2.9296860694885254, 'recall': 0.7929149617191463, 'precision': 0.7870110592469598, 'message': "The following patient(s) was/were not resampled back to the original resolution: ['CHUS085', 'CHUS030', 'CHGJ087', 'CHUP012', 'CHGJ074', 'CHUS076', 'CHGJ081', 'CHUM019', 'CHUS045', 'CHUS048', 'CHUS028', 'CHGJ018', 'CHUP009', 'CHUM001', 'CHMR020', 'CHMR014', 'CHUM022', 'CHUS064', 'CHGJ034', 'CHUP008', 'CHUM030', 'CHGJ090', 'CHUS081', 'CHUS087', 'CHUS056', 'CHGJ058', 'CHUS027', 'CHUP010', 'CHGJ028', 'CHUP007', 'CHUM036', 'CHGJ035', 'CHUM008', 'CHMR013', 'CHMR016', 'CHGJ008', 'CHMR005', 'CHUP003', 'CHUS049', 'CHUS077', 'CHUS007', 'CHMR024', 'CHUM060', 'CHUP021', 'CHGJ017', 'CHUM024', 'CHUM065', 'CHGJ048', 'CHGJ015', 'CHUM047', 'CHUM017', 'CHUS039', 'CHUM064', 'CHUM058', 'CHUS041', 'CHGJ073', 'CHU

100%|██████████| 224/224 [02:53<00:00,  1.29it/s]

{'dice_score': 0.7534715550724889, 'hausdorff_distance_95': 3.1431504149897123, 'recall': 0.7666546721436508, 'precision': 0.7969064679293646}
{'dice_score': 0.7534715550724889, 'hausdorff_distance_95': 3.1431504149897123, 'recall': 0.7666546721436508, 'precision': 0.7969064679293646, 'message': "The following patient(s) was/were not resampled back to the original resolution: ['CHUS085', 'CHUS030', 'CHGJ087', 'CHUP012', 'CHGJ074', 'CHUS076', 'CHGJ081', 'CHUM019', 'CHUS045', 'CHUS048', 'CHUS028', 'CHGJ018', 'CHUP009', 'CHUM001', 'CHMR020', 'CHMR014', 'CHUM022', 'CHUS064', 'CHGJ034', 'CHUP008', 'CHUM030', 'CHGJ090', 'CHUS081', 'CHUS087', 'CHUS056', 'CHGJ058', 'CHUS027', 'CHUP010', 'CHGJ028', 'CHUP007', 'CHUM036', 'CHGJ035', 'CHUM008', 'CHMR013', 'CHMR016', 'CHGJ008', 'CHMR005', 'CHUP003', 'CHUS049', 'CHUS077', 'CHUS007', 'CHMR024', 'CHUM060', 'CHUP021', 'CHGJ017', 'CHUM024', 'CHUM065', 'CHGJ048', 'CHGJ015', 'CHUM047', 'CHUM017', 'CHUS039', 'CHUM064', 'CHUM058', 'CHUS041', 'CHGJ073', 'CHU